# Importing tools

In [1]:
# pip install ortools
# pip install networkx
import networkx as nx
import time

# Importing libraries

In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

# Plotting the graph

In [3]:
def plot_graph(graph):
    g = nx.Graph()
    A = np.array(graph)
    g = nx.from_numpy_matrix(A, create_using=nx.Graph)

    pos = nx.spring_layout(g)
    labels = nx.get_edge_attributes(g,'weight')
    nx.draw(g, pos, edge_labels=labels, with_labels=True)

# Creating data models

In [4]:
# creating the data model for distance matrix
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 84, 72, 72, 56, 75, 9, 11, 14, 4, 8, 12, 10, 6
        ],
        [
            84, 0, 132, 157, 141, 148, 75, 99, 110, 79, 84, 89, 87, 85
        ],
        [
            72, 132, 0, 69, 79, 126, 66, 77, 74, 87, 123, 88, 80, 100
        ],
        [
            72, 157, 69, 0, 20, 70, 92, 76, 67, 71, 67, 64, 63, 67
        ],
        [
            56, 141, 79, 20, 0, 53, 60, 61, 47, 55, 51, 48, 51, 51
        ],
        [
            75, 148, 126, 70, 53, 0, 80, 60, 79, 66, 68, 67, 60, 62
        ],
        [
            9, 75, 66, 92, 60, 80, 0, 18, 18, 5, 10, 14, 13, 11
        ],
        [
            11, 99, 77, 76, 61, 60, 18, 0, 12, 12, 10, 14, 11,7
        ],
        [
            14, 110, 74, 67, 47, 79, 18, 12, 0, 14, 10, 14, 8, 9
        ],
        [
            4, 79, 87, 71, 55, 66, 5, 12, 14, 0, 5, 8, 8, 7
        ],
        [
            8, 84, 123, 67, 51, 68, 10, 10, 10, 5, 0, 5, 4, 3
        ],
        [
            12, 89, 88, 64, 48, 67, 14, 14, 14, 8, 5, 0, 5, 7
        ],
        [
            10, 87, 80, 63, 47, 60, 13, 11, 8, 8, 4, 5, 0, 4
        ],
        [
            6, 85, 100, 67, 51, 62, 11, 7, 9, 7, 3, 7, 4, 0
        ],
    ]
    data['demands'] = [0, 8, 11, 5, 6, 7, 9, 4, 14, 2, 1, 3, 10, 20]
    data['vehicle_capacities'] = [25, 25, 25, 25]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data

# Printing the solution

In [5]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    # Display dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(dropped_nodes)
    
    total_distance = 0
    total_load = 0
    max_route_distance = 0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    max_route_distance = max(route_distance, max_route_distance)
#     print('Maximum of the route distances: {}m'.format(max_route_distance))

# Main function

In [6]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        150,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
    # Allow to drop nodes.
    penalty = 3000
    for node in range(1, len(data['distance_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.SAVINGS)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')


if __name__ == '__main__':
    start_time = time.time()
    main()
    print("---------------")
    print("Total time taken :: %s seconds" % (time.time() - start_time))

Objective: 18255
Dropped nodes: 1
Route for vehicle 0:
 0 Load(0) ->  3 Load(5) ->  13 Load(25) ->  0 Load(25)
Distance of the route: 145m
Load of the route: 25

Route for vehicle 1:
 0 Load(0) ->  12 Load(10) ->  5 Load(17) ->  7 Load(21) ->  0 Load(21)
Distance of the route: 141m
Load of the route: 21

Route for vehicle 2:
 0 Load(0) ->  2 Load(11) ->  6 Load(20) ->  9 Load(22) ->  0 Load(22)
Distance of the route: 147m
Load of the route: 22

Route for vehicle 3:
 0 Load(0) ->  8 Load(14) ->  4 Load(20) ->  11 Load(23) ->  10 Load(24) ->  0 Load(24)
Distance of the route: 122m
Load of the route: 24

Total distance of all routes: 555m
Total load of all routes: 92
---------------
Total time taken :: 0.05358767509460449 seconds
